## Sekkyone - IPV6 Network Experiment - IoT Course 2020/2021

#### Radio settings

The following cells give us random value for *CHANNEL*, in order to avoid busy channels and possible collisions

In [1]:
%env SITE=saclay

env: SITE=saclay


In [10]:
!whoami

jovyan


In [6]:
import random
channel = random.randint(11, 26)
print('Use CHANNEL={}'.format(channel))

Use CHANNEL=16


In [7]:
print(channel)

16


In [ ]:
%env CHANNEL=11

## Experiment Submitting

In order to start our experiment, we will use, at the moment, the following configuration of nodes:
* n.1  a8 node  -> Its purpose is to provide us an MQTT-Bridge beetween nodes and IoT Corre
* n.1  m3 node  -> It will be our border router
* n.2+ m3 nodes -> They will run our application firmware

In [9]:
!iotlab-experiment submit -n riot_m3 -d 120 -l 4,archi=m3:at86rf231+site=$SITE -l 1,archi=a8:at86rf231+site=$SITE

usage: iotlab-experiment submit [-h] [-p] [-n NAME] -d DURATION
                                [-r RESERVATION] -l NODES_LIST
                                [-s SITE_ASSOCIATION] [--help-list]
                                [--help-site-association]
iotlab-experiment submit: error: argument -l/--list: Invalid arguments in experiment list '4,archi=m3:at86rf231+site=': Invalid empty value for property "site"


In [ ]:
!iotlab-experiment wait --timeout 60 --cancel-on-timeout

Use the following commands for additional info about the reserved nodes:

In [ ]:
!iotlab-experiment get -i <EXPERIMENT_ID> -p

In [ ]:
!iotlab-experiment get -i <EXPERIMENT_ID> -n

## Build and Flash firmwares

Open a Jupyter terminal (use `File > New > Terminal`) and run the following commands from it.

The following command needs to be run ONLY once

We will use minimum 3 distrinct terminals for building and flashing firmwares

### Border Router
Execute the following commands, by replacing the \<PLACEHOLDERS> with real info:

`source /opt/riot.source`

`cd RIOT`

`make ETHOS_BAUDRATE=500000 DEFAULT_CHANNEL=<CHANNEL> BOARD=iotlab-m3 -C examples/gnrc_border_router clean all`

`iotlab-node --flash examples/gnrc_border_router/bin/iotlab-m3/gnrc_border_router.elf -l saclay,m3,<ID>`

`sudo ethos_uhcpd.py m3-<ID> tap<K> 2001:660:3207:04c1::1/64`

Where:
1. `DEFAULT_CHANNEL` refers to the ENV variable added on top
2. `ID` is the M3 node number (ex: m3-1, ID is "1")
3. `K` is the tap interface number (ex: tap0, tap1...)

Always check that the interface and the channel are free

### MQTT-SN broker
We will use mosquitto.rsmb for communicating with the EC2 machine. Execute the following commands:

Create the following file inside the A8 folder:


`ssh root@node-a8-<ID>`


`cd A8`

`ip -6 -o addr show eth0` // TO GET THE IP Address of the MQTT broker, global link

`broker_mqtts config.conf`

### Sekkyone application firmware
For all the remaining reserved m3 nodes, execute the following commands:

`source /opt/riot.source`

`cd dev`

where dev is the folder where `main.c` and `makefile` files are contained

`make DEFAULT_CHANNEL=<CHANNEL> SERVER_ADDR=<MQTT_SERVER_ADDR> EMCUTE_ID=station_<K> BOARD=iotlab-m3 -C . clean all`

`iotlab-node --flash ./bin/iotlab-m3/sekkyone_mesh_network.elf -l saclay,m3,<ID>`

Eventually, nc to the node\[s\] to see if everything works fine:

`nc m3-<ID> 20000`

### Free up the resources

Once done, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop

## Performance Evaluation

In [ ]:
!unzip perf.zip

In [ ]:
!ls perf/tst3/266481/consumption

In [ ]:
import pandas

In [ ]:
dataset = pandas.read_csv("./perf/tst3/266481/consumption/m3_2.oml", sep='\t', skiprows=9, header=0, names=["timestamp_s", "timestamp_us", 'ign1', 'ign2', "power", "voltage", "current"])

In [ ]:
dataset

In [ ]:
ax = dataset.power.plot(figsize=(14,4), label='Power (W)')
ax.set_ylabel('Power (W)')
ax.set_xlabel('Time (ms)')

In [ ]:
ax = dataset.current.plot(figsize=(14,4), label='Current (mAh)')
ax.set_ylabel('Current (mAh)')
ax.set_xlabel('Time (ms)')

In [ ]:
ax = dataset.voltage.plot(figsize=(14,4), label='Voltage (V)')
ax.set_ylabel('Voltage (V)')
ax.set_xlabel('Time (ms)')

In [ ]:
!ls perf/tst1/266474/radio

In [ ]:
dataset2 = pandas.read_csv("./perf/tst3/266481/radio/m3_2.oml", sep='\t', skiprows=9, header=0, names=["timestamp_s", "timestamp_us", "channel", "rssi"])

In [ ]:
dataset2

In [ ]:
ax = dataset2.rssi.plot(figsize=(14,4), label='RSSI (dBm)')
ax.set_ylabel('RSSI (dBm)')
